<a href="https://colab.research.google.com/github/jfraska/123200171_123200164/blob/master/smart_cowshed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import io
from PIL import Image
import random
from shutil import copyfile

In [ ]:
# Get dataset
os.environ['KAGGLE_USERNAME'] = "alifianzulfaani"
os.environ['KAGGLE_KEY'] = "4b8aa99d8f0cfab2b85c684ad6864489"
!kaggle datasets download -d alifianzulfaani/cowshed-data

100% 26.0M/26.0M [00:01<00:00, 25.5MB/s]
100% 26.0M/26.0M [00:01<00:00, 15.1MB/s]


## Aleep

In [ ]:
# Extract dataset
from zipfile import ZipFile
file_name = "/content/cowshed-data.zip"
zp = ZipFile(file_name, 'r')
zp.extractall(path='/content/')
print('done')

done


In [ ]:
# Split folder
!pip install split-folders
import splitfolders

splitfolders.ratio('/content/Dataset',
                   output="/content/used-cowshed-data", seed=1337, ratio=(0.7,0.2,0.1))

Copying files: 158 files [00:00, 1775.21 files/s]


In [ ]:
# Check the name of classes
os.listdir('/content/used-cowshed-data/train')
os.listdir('/content/used-cowshed-data/val')
os.listdir('/content/used-cowshed-data/test')

['air_liur_sehat',
 'kaki_sehat',
 'gusi_sehat',
 'air_liur_sakit',
 'lidah_sakit',
 'lidah_sehat',
 'kaki_sakit',
 'gusi_sakit']

In [ ]:
# Set the name of directory
base_dir = '/content/used-cowshed-data'
train_dir = os.path.join(base_dir, 'train/')
val_dir = os.path.join(base_dir, 'val/')
test_dir = os.path.join(base_dir, 'test/')

train_aug_dir = os.path.join(base_dir, 'aug/')
# val_aug = '/content/augmented/validation/'
# test_aug = '/content/augmented/test/'
print('done')

done


In [ ]:
# Image augmentation
IMG_WIDTH = 150
IMG_HEIGHT = 150
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='constant'
    )

def aug_img(input_dir, output_dir, nums):
    itr_num = nums
    itr = 0
    for each_data in os.listdir(input_dir):
        dataset = []
        image_directory = input_dir+each_data+'/'
        dataset = []

        my_images = os.listdir(image_directory)
        for i, image_name in enumerate(my_images):
            if (image_name.split('.')[1] == 'jpg' or 'jpeg'):
                image = io.imread(image_directory + image_name)
                image = Image.fromarray(image, 'RGB')
                image = image.resize((IMG_WIDTH,IMG_HEIGHT))
                dataset.append(np.array(image))

        x = np.array(dataset)
        aug_dir = output_dir+each_data
        os.makedirs(aug_dir)

        i = 1
        for batch in datagen.flow(x, batch_size=16, save_to_dir=aug_dir,
                                  save_prefix='aug'+image_name,
                                  save_format='png'):
            i += 1
            if i > itr_num[itr]:
                break

        itr +=1
        print('images are: '.format(i)+str(len(os.listdir(aug_dir))))
    print()

num_train = [15, 13, 15, 10, 10, 10 ,10, 10]
# num_val = [0,0,0,0]
# num_test = [0,0,0,0]

aug_img(train_dir, train_aug_dir, num_train)
# aug_img(validation_dir, val_aug, num_val)
# aug_img(test_dir, test_aug, num_test)

images are: 163
images are: 208
images are: 105
images are: 90
images are: 99
images are: 100
images are: 120
images are: 130



In [ ]:
# Processing Data
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(rescale=1. /255)
train_generator = train_datagen.flow_from_directory(
        train_aug_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

validation_datagen = ImageDataGenerator(
                    rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

test_datagen = ImageDataGenerator(
                    rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 1015 images belonging to 8 classes.
Found 29 images belonging to 8 classes.
Found 22 images belonging to 8 classes.


In [ ]:
# Count of training data after augmentation
print("Total Data Augmentasi: ", sum([len(files) for r, d, files in os.walk('/content/used-cowshed-data/aug')]))
print("Total Data Validasi: ", sum([len(files) for r, d, files in os.walk('/content/used-cowshed-data/val')]))

Total Data Augmentasi:  1015
Total Data Validasi:  29


In [ ]:
# Index class
test_generator.class_indices

{'air_liur_sakit': 0,
 'air_liur_sehat': 1,
 'gusi_sakit': 2,
 'gusi_sehat': 3,
 'kaki_sakit': 4,
 'kaki_sehat': 5,
 'lidah_sakit': 6,
 'lidah_sehat': 7}

## EfficientNet versi B0

In [ ]:
!pip install -U efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.5 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
import efficientnet.keras as efn

base_model = efn.EfficientNetB7(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

258434480/258434480 [==============================] - 5s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

# Add a final softmax layer with 8 node for classification output
predictions = Dense(8, activation="softmax")(x)
model_final = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
model_final.compile(tf.keras.optimizers.legacy.RMSprop(learning_rate=0.0001, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Use of callbacks to prevents overfitting and stops training once accuracy is met
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_loss') < 0.1):
      print("\nLoss di bawah 10%, hentikan training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
eff_history = model_final.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 32, epochs = 10, callbacks=[callbacks])

Epoch 1/10
32/32 [==============================] - 358s 10s/step - loss: 2.5471 - accuracy: 0.5034 - val_loss: 1.2869 - val_accuracy: 0.7586
Epoch 2/10


## Evaluasi Metrik

In [ ]:
# Loss train & validation
plt.plot(eff_history.history['loss'], label='Training Loss')
plt.plot(eff_history.history['val_loss'], label='Validation Loss')
plt.title('Loss Plot')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
# Accuracy train & validation
plt.plot(eff_history.history['accuracy'], label='Training Accuracy')
plt.plot(eff_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Plot')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))

  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)
  output_class = np.argmax(classes)
  print(fn)
  if output_class==0:
      print('air_liur_sakit')
  elif output_class==1:
      print('air_liur_sehat')
  elif output_class==2:
      print('gusi_sakit')
  elif output_class==3:
      print('gusi_sehat')
  elif output_class==4:
      print('kaki_sakit')
  elif output_class==5:
      print('kaki_sehat')
  elif output_class==6:
      print('lidah_sakit')
  elif output_class==7:
      print('lidah_sehat')